In [131]:
import imp
import numpy as np
import pandas as pd
import re
import pickle as pkl
import xlrd

In [ ]:
from IPython.core.display import HTML
display(HTML(df.to_html()))

In [382]:
d = imp.load_source('dashboard', 'dashboard.py')
g = imp.load_source('gsync', 'modules/gCanvas.py')
c = imp.load_source('conf', 'config/conf.py')
r = imp.load_source('report', 'modules/reader.py')
s = imp.load_source('patch', 'modules/patcher.py')
t = imp.load_source('triggers', 'config/triggers.py')

In [345]:
df = r.get_report('data/report/all.xlsx', 0)

In [346]:
df = df[df.channel != 'Ручные рассылки']

In [244]:
known_names = []
for key, value in t.triggers.items():
    for df_name in value.keys():
        known_names.append(df_name)

In [245]:
unknown_names = []
for name in list(df.name.unique()):
    if name not in known_names:
        unknown_names.append(name)

In [266]:
from gspread_dataframe import *
from gspread_formatting import *

In [407]:
conn = g.Connection("new test dashboard")

In [408]:
ws = conn.get_sheet_by_name("3. Триггеры — стат. нед.")
gdf = get_as_dataframe(ws)

In [409]:

#, index_col=0

In [468]:
gdf[gdf.columns[0]][gdf[gdf.columns[0]] == 'DOI Черная пятница'].index

Int64Index([379], dtype='int64')

In [464]:
reindex_outlay(gdf, 'seasonal')

<function str.index>

In [437]:
gdf[gdf.columns[1]]

0         номер недели
1                 даты
2      Трафик на сайте
3          Регистраций
4                Email
            ...       
468                NaN
469                NaN
470                NaN
471                NaN
472                NaN
Name:  , Length: 473, dtype: object

In [ ]:
, dataframe.index[dataframe.columns[1] == trigger]

In [470]:
def reindex_outlay(dataframe):
    for channel in t.triggers.keys():
        trigger = list(t.triggers[channel].values())[0]
        init[channel] = gdf[gdf.columns[0]][gdf[gdf.columns[0]] == trigger].index


In [475]:
def put_dictionary(filename, obj):
    with open(filename, 'wb') as f:
        pkl.dump(obj, f, protocol=pkl.HIGHEST_PROTOCOL)

In [476]:
put_dictionary('config/dictionaries/outlay3.pkl',init)

In [473]:
del init['start']

In [477]:
gdf.iloc[31:379]

,,Описание триггера,Условия отправки,Ссылка на визуал,август,Unnamed: 5,Unnamed: 6,Unnamed: 7,сентябрь,Unnamed: 9,...,Unnamed: 15,ноябрь,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,декабрь,Unnamed: 22,Unnamed: 23,Unnamed: 24
31,Брошенная корзина email,NaN,Положил товар в корзину \nи\nНе совершил покуп...,https://perekrestok.mindbox.ru/campaigns/4/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,Брошенных корзин,NaN,NaN,NaN,11199,0,7511,40176,42429,44804,...,38975,38816,40668,42671,61557,121045,50437,50841,NaN,NaN
33,Доставлено,NaN,NaN,NaN,2967,2458,2958,3663,3040,3673,...,1733,2781,3983,4236,8186,12360,5664,5869,NaN,NaN
34,Процент отработанных корзин,NaN,NaN,NaN,=E35/E34,NaN,=G35/G34,=H35/H34,=I35/I34,=J35/J34,...,=P35/P34,=Q35/Q34,=R35/R34,=S35/S34,=T35/T34,=U35/U34,=V35/V34,=W35/W34,NaN,NaN
35,Открытия,NaN,NaN,NaN,1313,1052,1162,1412,1165,1324,...,654,928,1360,1487,3014,4547,2047,2123,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,Доход по атрибуции MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
375,Покупок по атрибуции MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
376,Средний чек по атрибуции,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,=iferror(U376/U377;0),NaN,NaN,NaN,NaN
377,Конверсия в заказы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,=IFERROR(U377/U370;0),NaN,NaN,NaN,NaN


In [375]:
#tmp = s.check_new_campaigns(unknown_names)
init = s.get_dictionary('config/dictionaries/outlay3.pkl')
email = gdf.iloc[init['start']:init['email']]
wp = gdf.iloc[init['email']:init['wp']]
seasonal = gdf.iloc[init['wp']:init['seasonal']]
sms = gdf.iloc[init['seasonal']:init['sms']]

In [384]:
online_attribution = t.online_attr

In [387]:
del online_attribution['subject'], online_attribution['preview_url'] 
s.put_dictionary('config/dictionaries/online_attr.pkl', online_attribution)
online_attribution = s.get_dictionary('config/dictionaries/online_attr.pkl')

In [360]:
def merge_dictionaries(dict1, list_ds):
    for d in list_ds:
        dict1.update(d)
    return dict1

ds = [t.triggers['email'], 
      t.triggers['wp'],
      t.triggers['seasonal'], 
      t.triggers['sms']]

In [362]:
cumul_d = {}
cumul_d = merge_dictionaries(cumul_d, ds)

In [ ]:
for channel in 

In [365]:
#df.name.map(cumul_d)

In [405]:
wp

,Описание триггера,Условия отправки,Ссылка на визуал,август,Unnamed: 5,Unnamed: 6,Unnamed: 7,сентябрь,Unnamed: 9,Unnamed: 10,...,Unnamed: 15,ноябрь,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,декабрь,Unnamed: 22,Unnamed: 23,Unnamed: 24
,,,,,,,,,,,,,,,,,,,,,
Брошенная корзина Web-push,NaN,Последние 24 час не покупал. Неизвестных проду...,https://perekrestok.mindbox.ru/campaigns/4/mai...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Брошенных корзин,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Доставлено,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,431,807,1109,1239,1718,2559,1844,1995,NaN,NaN
Процент отработанных корзин,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Открытия,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,315,615,806,947,1322,1984,1405,1526,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Доход по атрибуции MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
Покупок по атрибуции MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
Средний чек по атрибуции,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,=iferror(U376/U377;0),NaN,NaN,NaN,NaN


In [ ]:
def append_campaign(dataframe, channel):
    for trigger, campaign in t.triggers[channel].items():
        dataframe = dataframe.append(pd.Series(name = campaign))
        for key, attribution in online_attribution.items():
            dataframe = dataframe.append(pd.Series(name = attribution))
            dataframe.iloc[-1, self.limit] = mb_df.loc[df.name == trigger, key]

In [388]:
def extend_dataframe(self, mb_df):
    """Inserts new rows for new campaigns to gspread-dataframe
    -> mb_df: dataframe from MindBox report
    """
    init = s.get_dictionary('config/dictionaries/outlay3.pkl')
    email = gdf.iloc[init['start']:init['email']]
    wp = gdf.iloc[init['email']:init['wp']]
    seasonal = gdf.iloc[init['wp']:init['seasonal']]
    sms = gdf.iloc[init['seasonal']:init['sms']]
    
    channel_dfs = [email, wp, seasonal, sms]
    del init['start']
    
    result_df = pd.DataFrame()
    for channel, dataframe in dict(zip(init.values(), channel_dfs)):
        for trigger, campaign in t.triggers[channel].items():
            dataframe = dataframe.append(pd.Series(name = campaign))
            for key, attribution in online_attribution.items():
                dataframe = dataframe.append(pd.Series(name = attribution))
                dataframe.iloc[-1, self.limit] = mb_df.loc[df.name == trigger, key]

        result_df = result_df.append(dataframe)
    return result_df

48    6507
Name: delivered, dtype: int64
48    2366
Name: opened, dtype: int64
48    0.363608
Name: open_rate, dtype: float64
48    600
Name: clicked, dtype: int64
48    0.092208
Name: click_rate, dtype: float64
48    0.253593
Name: CTR, dtype: float64
48    5
Name: unfollowed, dtype: int64
48    0.000768
Name: unfollow_rate, dtype: float64
48    713121
Name: revenue, dtype: int64
48    173
Name: orders, dtype: int64
48    4122.089711
Name: avg_bill, dtype: float64
48    0.026587
Name: order_conversion, dtype: float64


In [395]:
email

,Описание триггера,Условия отправки,Ссылка на визуал,август,Unnamed: 5,Unnamed: 6,Unnamed: 7,сентябрь,Unnamed: 9,Unnamed: 10,...,Unnamed: 15,ноябрь,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,декабрь,Unnamed: 22,Unnamed: 23,Unnamed: 24
,,,,,,,,,,,,,,,,,,,,,
Брошенная корзина email,NaN,Положил товар в корзину \nи\nНе совершил покуп...,https://perekrestok.mindbox.ru/campaigns/4/mai...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Брошенных корзин,NaN,NaN,NaN,11199,0,7511,40176,42429,44804,45207,...,38975,38816,40668,42671,61557,121045,50437,50841,NaN,NaN
Доставлено,NaN,NaN,NaN,2967,2458,2958,3663,3040,3673,3767,...,1733,2781,3983,4236,8186,12360,5664,5869,NaN,NaN
Процент отработанных корзин,NaN,NaN,NaN,=E35/E34,NaN,=G35/G34,=H35/H34,=I35/I34,=J35/J34,=K35/K34,...,=P35/P34,=Q35/Q34,=R35/R34,=S35/S34,=T35/T34,=U35/U34,=V35/V34,=W35/W34,NaN,NaN
Открытия,NaN,NaN,NaN,1313,1052,1162,1412,1165,1324,1379,...,654,928,1360,1487,3014,4547,2047,2123,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Средний чек по атрибуции,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,=U300/U301,0,0,NaN,NaN
Конверсия в заказы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,=IFERROR(U301/U292;0),=IFERROR(V301/V292;0),=IFERROR(W301/W292;0),NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [402]:
email.iloc[-1,-2]

nan

{'delivered': 'Доставлено',
 'opened': 'Открытия',
 'open_rate': 'OR',
 'clicked': 'Кликов',
 'click_rate': 'CR',
 'ctor': 'CTR',
 'unfollowed': 'Отписок',
 'unfollow_rate': 'UR',
 'revenue': 'Доход по атрибуции MB',
 'orders': 'Покупок по атрибуции MB',
 'avg_bill': 'Средний чек по атрибуции',
 'order_conversion': 'Конверсия в заказы'}

In [347]:
tmp

{'email': 6, 'seasonal': 2, 'wp': 0}

In [147]:
#set_with_dataframe(conn.get_sheet_by_name("Лист10"), df)

In [193]:
vals = df[df.columns[0]].unique()

In [194]:
known_names = []
for key, value in t.triggers.items():
    for df_name in value.values():
        known_names.append(df_name)

In [195]:
boolcol = []
for val in df[df.columns[0]]:
    boolcol.append(val in known_names)

In [196]:
df.loc[boolcol]

,,Описание триггера,Условия отправки,Ссылка на визуал,август,Unnamed: 5,Unnamed: 6,Unnamed: 7,сентябрь,Unnamed: 9,...,Unnamed: 15,ноябрь,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,декабрь,Unnamed: 22,Unnamed: 23,Unnamed: 24
31,Брошенная корзина email,NaN,Положил товар в корзину \nи\nНе совершил покуп...,https://perekrestok.mindbox.ru/campaigns/4/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,Welcome 1 new email,NaN,"Подписался через Личный кабинет на сайте, дейс...",https://perekrestok.mindbox.ru/campaigns/5/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Брошенный просмотр товара email,NaN,Просматривал от 4 товаров и ни чего не добавил...,https://perekrestok.mindbox.ru/campaigns/4/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Брошенный просмотр категории email,NaN,Не смотрел товары и Не добавлял товары в корзи...,https://perekrestok.mindbox.ru/campaigns/4/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Неделя до ДР email,NaN,Дата дня рождения заполнена и до него 7 дней,https://perekrestok.mindbox.ru/campaigns/8/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,Реактивация email,NaN,Не открывал письма последние 30 дней,https://perekrestok.mindbox.ru/campaigns/6/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,Товар появился в наличии email,NaN,Продукт из списка Избранных стал доступен. Пос...,https://perekrestok.mindbox.ru/campaigns/11/ma...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,Напоминание о промокоде А email,NaN,Отправить через 3 дня после Отправки Welcome....,https://perekrestok.mindbox.ru/campaigns/5/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,Напоминание о промокоде В email,NaN,Отправить через 3 дня после Отправки Welcome. ...,https://perekrestok.mindbox.ru/campaigns/5/mai...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,Отток Активные email,NaN,"Было от 3 покупок, последние 45 дней не покупа...",https://perekrestok.mindbox.ru/campaigns/12/ma...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
outlay = {'email':302,
          'wp':378,
          'seasonal':434,
          'sms':437}

In [213]:
s.put_dictionary('config/dictionaries/outlay3.pkl', outlay)

In [72]:
df = pd.read_excel('data/report/all.xlsx')
ind = df.loc[df.iloc[:,0] == 'Бренд'].index[0]
df = df.iloc[ind+2:, 4:-1]
header = ['name', 'channel', 'subject', 'preview_url',
          'sent', 'delivered', 'deliver_rate', 'opened',
          'open_rate', 'clicked', 'click_rate', 'CTR',
          'unfollowed', 'unfollow_rate', 'revenue', 'orders',
          'avg_bill', 'order_conversion']
df.columns = header
df[['sent', 'delivered', 'opened', 'clicked',
    'unfollowed', 'revenue', 'orders']] = \
df[['sent', 'delivered', 'opened', 'clicked',
    'unfollowed', 'revenue', 'orders']].astype('int64')
df[['deliver_rate', 'open_rate', 'click_rate', 'CTR',
    'unfollow_rate', 'avg_bill', 'order_conversion']] = \
df[['deliver_rate', 'open_rate', 'click_rate', 'CTR',
    'unfollow_rate', 'avg_bill', 'order_conversion']].astype('float64')
#return df


In [63]:
names = df[(df.name != '(Для тестов)') & (df.sent != 0)]['name'].unique()

array(['Триггер 1. Брошенная корзина', 'Триггер_2_Брошенный просмотр',
       'Триггер_4 "Брошенный просмотр категории"',
       'WP "Брошенная корзина"',
       'Триггер 17. Брошенная корзина с оффером',
       'Welcome 1. Приветствие и промокод NEW',
       'Триггер_3_Напоминание о промокоде A',
       'Триггер_3_Напоминание о промокоде В',
       'Триггер_12_Напоминание о промокоде NEW_2',
       'Триггер_13_Напоминание о промокоде NEW_3',
       'Ваш промокод уже здесь! (Facebook Leads)', 'NY landing 2020 DOI',
       'NY landing 2020 DOI promocode', 'Триггер_5 "Реактивация"',
       'Триггер_7_Товар появился в наличии', 'Email Отток Активные',
       'Email trig_water_reminder', 'WP water_reminder',
       'Триггер_14_ "Онлайн дешевле"', 'Триггер Мираторг + Tefal',
       'DOI Ёлки', 'Ёлки Спасибо', 'Копия Ёлка SMS',
       'Письмо с промокодом Babyblog (для новых)', 'DOI BabyBlog',
       'Персональный промокод на скидку — в письме! 🤤', 'SMS с опросом',
       'SMS без оффера', '

In [ ]:
def prepare_campaign_names(names):
    if 'Триггер' in name:
        txt = re.findall(r'[А-я]+', data)

In [34]:
df.iloc[ind+2:]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
48,Perekrestok,Брошенные действия,без тэга,2019-07-05 11:50:15,Триггер 1. Брошенная корзина,Автоматические рассылки,@{if IsEmpty(Recipient.OnlyStandardFirstName)}...,https://email-services.mindbox.ru/v1/projects/...,6511,6507,...,600,0.0922084,0.253593,5,0.000768403,713122,173,4122.09,0.0265868,109.593
49,Perekrestok,Брошенные действия,без тэга,2019-07-11 13:50:41,Триггер_2_Брошенный просмотр,Автоматические рассылки,Хотите добавить товары в свою корзину?,https://email-services.mindbox.ru/v1/projects/...,314,314,...,20,0.0636943,0.15873,1,0.00318471,16734.3,4,4183.58,0.0127389,53.294
50,Perekrestok,Брошенные действия,без тэга,2019-08-12 21:43:57,"Триггер_4 ""Брошенный просмотр категории""",Автоматические рассылки,Вы что-то искали? 🤔,https://email-services.mindbox.ru/v1/projects/...,2516,2513,...,132,0.0525269,0.169666,4,0.00159172,103441,25,4137.65,0.00994827,41.1625
51,Perekrestok,Брошенные действия,без тэга,2019-09-18 14:10:19,(Для тестов),Автоматические рассылки,Тест,https://email-services.mindbox.ru/v1/projects/...,2,2,...,0,0,0,0,0,0,0,0,0,0
52,Perekrestok,Брошенные действия,без тэга,2019-09-27 12:49:21,"WP ""Брошенная корзина""",Автоматические рассылки,Без темы,(blank),2392,2392,...,84,0.0351171,0.0457018,0,0,53795.1,14,3842.51,0.00585284,22.4896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Perekrestok,Черная пятница 2019,без тэга,2019-11-08 16:21:57,DOI Черная пятница Спасибо,Автоматические рассылки,Спасибо за регистрацию!,https://email-services.mindbox.ru/v1/projects/...,0,0,...,0,0,0,0,0,0,0,0,0,0
112,Perekrestok,Интерактивный тест,без тэга,2019-11-15 12:52:11,DOI Интерактивный тест,Автоматические рассылки,Ваш промокод в письме!,https://email-services.mindbox.ru/v1/projects/...,0,0,...,0,0,0,0,0,0,0,0,0,0
113,Perekrestok,Интерактивный тест,без тэга,2019-11-15 14:16:17,DOI Интерактивный тест Спасибо,Автоматические рассылки,Ваш промокод в письме!,https://email-services.mindbox.ru/v1/projects/...,0,0,...,0,0,0,0,0,0,0,0,0,0
114,Perekrestok,Ивенты,без тэга,2019-12-05 17:51:21,Начинаем подготовку к Новому году! 😍🌟,Ручные рассылки,Начинаем подготовку к Новому году! 😍🌟,https://email-services.mindbox.ru/v1/projects/...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
#df = pd.read_excel('data/report/all.xlsx', sheet_name='Свод. данные (online)')
df = get_report('data/report/all.xlsx', 'Свод. данные (online + offline)')

In [72]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [8]:
s = """brand 	campaign 	tag 	date 	name 	channel 	subject 	preview_url 	sent 	delivered 	... 	clicked 	click_rate 	ctor 	unfollowed 	unfollow_rate 	revenue 	orders 	avg_bill 	order_conversion 	revenue_per_user"""

In [10]:
' '.join(s.split(' 	'))

'brand campaign tag date name channel subject preview_url sent delivered ... clicked click_rate ctor unfollowed unfollow_rate revenue orders avg_bill order_conversion revenue_per_user'

In [78]:
df.columns

Index(['brand', 'campaign', 'tag', 'date', 'name', 'channel', 'subject',
       'preview_url', 'sent', 'delivered', 'delivery_rate', 'opened',
       'open_rate', 'clicked', 'click_rate', 'ctor', 'unfollowed',
       'unfollow_rate', 'revenue', 'orders', 'avg_bill', 'order_conversion',
       'revenue_per_user'],
      dtype='object')

In [77]:
df[(df.sent != 0) & (df.channel == 'Ручные рассылки')]

,brand,campaign,tag,date,name,channel,subject,preview_url,sent,delivered,...,clicked,click_rate,ctor,unfollowed,unfollow_rate,revenue,orders,avg_bill,order_conversion,revenue_per_user
79,Perekrestok,A/B тест для Активных,без тэга,2019-12-16 19:01:20,SMS без оффера,Ручные рассылки,Без темы,(blank),227,203,...,0,0.000000,0.000000,0,0.000000,0.00,0,0.000000,0.000000,0.000000
80,Perekrestok,A/B тест для Активных,без тэга,2019-12-16 19:01:20,SMS с оффером,Ручные рассылки,Без темы,(blank),233,205,...,0,0.000000,0.000000,0,0.000000,0.00,0,0.000000,0.000000,0.000000
83,Perekrestok,Регулярные рассылки,без тэга,2019-12-17 19:00:50,171219_elki,Ручные рассылки,🌲Пушистые елки из России и Дании 🌲,https://email-services.mindbox.ru/v1/projects/...,200642,198025,...,5507,0.027810,0.197193,251,0.001268,1713695.83,393,4360.549186,0.001985,8.653937
84,Perekrestok,Регулярные рассылки,без тэга,2019-12-18 17:31:01,18122019_🎁Самое главное в подарке — это…,Ручные рассылки,🎁Самое главное в подарке — это…,https://email-services.mindbox.ru/v1/projects/...,200958,198561,...,5857,0.029497,0.159491,418,0.002105,2903501.53,665,4366.167714,0.003349,14.622718
85,Perekrestok,Регулярные рассылки,без тэга,2019-12-18 19:38:30,sms beauty_181219,Ручные рассылки,Без темы,(blank),21619,21189,...,0,0.000000,0.000000,0,0.000000,0.00,0,0.000000,0.000000,0.000000
105,Perekrestok,NY 2020,без тэга,2019-12-23 19:51:03,Мы снизили цены на елки! Скидки до 50%,Ручные рассылки,Мы снизили цены на елки! Скидки до 50%,https://email-services.mindbox.ru/v1/projects/...,202931,200573,...,5941,0.029620,0.166597,249,0.001241,1447405.40,368,3933.166848,0.001835,7.216352


,brand,campaign,tag,date,name,channel,subject,preview_url,sent,delivered,delivery_rate,opened,open_rate,clicked,click_rate,ctor,unfollowed,unfollow_rate,revenue,orders,avg_bill,order_conversion,revenue_per_user
48,Perekrestok,Брошенные действия,без тэга,2019-07-05 11:50:15,Триггер 1. Брошенная корзина,Автоматические рассылки,"@{if IsEmpty(Recipient.OnlyStandardFirstName)} @{set theme1 = ""Вы готовы завершить заказ?""}@{set theme2 = ""Пожалуйста, завершите заказ""}@{set theme3 = ""Перейдите в корзину и завершите заказ""}@{ else }@{set theme1 = Recipient.OnlyStandardFirstName & "", вы готовы завершить заказ?""}@{set theme2 = Recipient.OnlyStandardFirstName & "", пожалуйста, завершите заказ""}@{set theme3 = Recipient.OnlyStandardFirstName & "", перейдите в корзину и завершите заказ""}@{ end if }${ChooseRandomText(""Ваши товары ждут в корзине"",""Заберите их, пожалуйста, к себе домой!"",""В корзине вас ждут товары, которые хотят попасть домой"",""Все решено?"",""Хороший выбор!"",""Не откладывайте на завтра то, что можно сделать сегодня!"",""Посмотрите, вы ничего не забыли?"",theme1, theme2, theme3)}",https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=2305843010287435776&hash=AB1563AC2DFD3E45AFCD26E1A400232D8023C0D6&format=Html,6511,6507,0.999386,2366,0.363608,600,0.092208,0.253593,5,0.000768,713121.52,173,4122.089711,0.026587,109.592980
49,Perekrestok,Брошенные действия,без тэга,2019-07-11 13:50:41,Триггер_2_Брошенный просмотр,Автоматические рассылки,Хотите добавить товары в свою корзину?,https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=-6917529022272372736&hash=DFFCF1A1FCCEC05252D7DC651C35361FD655A6BD&format=Html,314,314,1.000000,126,0.401274,20,0.063694,0.158730,1,0.003185,16734.33,4,4183.582500,0.012739,53.294045
50,Perekrestok,Брошенные действия,без тэга,2019-08-12 21:43:57,"Триггер_4 ""Брошенный просмотр категории""",Автоматические рассылки,Вы что-то искали? 🤔,https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=3458764520799862784&hash=BB4380B55D6D9DDDB15EB123E7DFF6552750C75B&format=Html,2516,2513,0.998808,778,0.309590,132,0.052527,0.169666,4,0.001592,103441.27,25,4137.650800,0.009948,41.162463
51,Perekrestok,Брошенные действия,без тэга,2019-09-18 14:10:19,(Для тестов),Автоматические рассылки,Тест,https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=-7782220148445806592&hash=56FC83201517CA8A8D3702694E29B188B0B4CEAD&format=Html,2,2,1.000000,2,1.000000,0,0.000000,0.000000,0,0.000000,0.00,0,0.000000,0.000000,0.000000
52,Perekrestok,Брошенные действия,без тэга,2019-09-27 12:49:21,"WP ""Брошенная корзина""",Автоматические рассылки,Без темы,(blank),2392,2392,1.000000,1838,0.768395,84,0.035117,0.045702,0,0.000000,53795.13,14,3842.509286,0.005853,22.489603
53,Perekrestok,Брошенные действия,без тэга,2019-11-27 10:30:13,Триггер 17. Брошенная корзина с оффером,Автоматические рассылки,Дарим скидку на заказ из вашей корзины!,https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=8430738510121533440&hash=73C65FD1637E497507234D5ABBA4FEFCBB8C5793&format=Html,322,321,0.996894,163,0.507788,60,0.186916,0.368098,0,0.000000,140464.58,24,5852.690833,0.074766,437.584361
54,Perekrestok,DOI и Welcome,без тэга,2019-08-12 20:06:14,Welcome 1. Приветствие и промокод NEW,Автоматические рассылки,Промокод на первые три заказа,https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=-3458764511136186368&hash=CA791D417BBFBE17FBA2519768E1FBEEBA4BE8F3&format=Html,3436,3432,0.998836,2373,0.691434,531,0.154720,0.223767,6,0.001748,550227.25,150,3668.181667,0.043706,160.322625
55,Perekrestok,DOI и Welcome,без тэга,2019-08-13 11:56:58,Триггер_3_Напоминание о промокоде A,Автоматические рассылки,Не забудьте про скидку в 500 ₽,https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=-5764607521423622144&hash=E675092E942E94D477F5CF4138C413543D4ECB2F&format=Htm

In [45]:
df[(df.channel == 'Ручные рассылки') & df.sent != 0]

,brand,campaign,tag,date,name,channel,subject,preview_url,sent,delivered,...,clicked,click_rate,ctor,unfollowed,unfollow_rate,revenue,orders,avg_bill,order_conversion,revenue_per_user
79,Perekrestok,A/B тест для Активных,без тэга,2019-12-16 19:01:20,SMS без оффера,Ручные рассылки,Без темы,(blank),227,203,...,0,0.00000,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
80,Perekrestok,A/B тест для Активных,без тэга,2019-12-16 19:01:20,SMS с оффером,Ручные рассылки,Без темы,(blank),233,205,...,0,0.00000,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
85,Perekrestok,Регулярные рассылки,без тэга,2019-12-18 19:38:30,sms beauty_181219,Ручные рассылки,Без темы,(blank),21619,21189,...,0,0.00000,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
105,Perekrestok,NY 2020,без тэга,2019-12-23 19:51:03,Мы снизили цены на елки! Скидки до 50%,Ручные рассылки,Мы снизили цены на елки! Скидки до 50%,https://email-services.mindbox.ru/v1/projects/...,202931,200573,...,5941,0.02962,0.166597,249,0.001241,1447405.4,368.0,3933.166848,0.001835,7.216352


In [15]:
df.iloc[df.loc[df.iloc[:,0] == 'Бренд'].index[0]]

Unnamed: 0                               Бренд
Unnamed: 1                            Кампания
Unnamed: 2                                 Тэг
Unnamed: 3                Дата старта рассылки
Unnamed: 4                   Название рассылки
Unnamed: 5                               Канал
Unnamed: 6                         Тема письма
Unnamed: 7                     Ссылка на макет
Unnamed: 8                    Отправлено писем
Unnamed: 9                    Доставлено писем
Unnamed: 10                     % Доставленных
Unnamed: 11                Количество открытий
Unnamed: 12                          Open rate
Unnamed: 13                  Количество кликов
Unnamed: 14                         Click rate
Unnamed: 15                             CTOR  
Unnamed: 16                  Запрошена отписка
Unnamed: 17              % Запросов на отписку
Unnamed: 18                 Доход по атрибуции
Unnamed: 19    Количество заказов по атрибуции
Unnamed: 20           Средний чек по атрибуции
Unnamed: 21  

46

In [4]:
df = df[df.channel != 'Ручные рассылки']
df = df[['name', 'sent', 'delivered', 'opened', 'clicked', 'unfollowed', 'revenue', 'orders']]

In [26]:
df = df[df.sent!=0]

In [76]:
#res = check_seasonality(list(df.name.loc[~df.name.map(t.triggers).notna()]))

In [74]:
res

['NY landing 2020 DOI', 'NY landing 2020 DOI promocode']

 * Принципиально ли их иметь разбитыми на категории
 * Обработка сезонных триггеров

In [72]:
def check_seasonality(campaigns):
    maxlen = len(campaigns)
    print("""Проверьте, является ли компания сезонной для {n} новых;
    Нажмите только 'Enter', если компания не является сезонной
    Введите 1, если компания является сезонной
    Введите 0 для сброса ввода и возврату к первому значению""".format(n=maxlen))
    res = []
    i = 0
    while(i != maxlen):
        v = input(campaigns[i]+': ')
        if v == '':
             i+=1
        elif v == '1':
            res.append(campaigns[i]); i+=1
        elif v == '0':
            return check_seasonality(campaigns)
        else:
            i -= 1
            print("Повторите ввод в согласии с инструкцией\n")            
    return res

In [ ]:
check_seasonality

In [3]:
%%time
!python dashboard.py -p 1 -r "data/report.xlsx" -o 'data/orders.csv' -c 'data/client_groups.f'

dashboard.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
CPU times: user 112 ms, sys: 12 ms, total: 124 ms
Wall time: 5.96 s
